In [90]:
import pandas as pd
import numpy as np
import pickle
from cartopy import crs
from bokeh.models import ColorBar, ColumnDataSource, HoverTool, Select,  FixedTicker, NumeralTickFormatter, CustomJS
import bokeh.palettes
from bokeh.transform import linear_cmap
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.plotting import figure, save
from bokeh.io import output_file, show, output_notebook 
from bokeh.layouts import column

In [135]:
file = open("gdf_tract.pickle",'rb')
gdf_tract = pickle.load(file)
file.close()

file = open("la_hospital_capacity.pickle",'rb')
la_hospital_capacity = pickle.load(file)
file.close()

file = open("la_tract_income.pickle",'rb')
la_tract_income = pickle.load(file)
file.close()

fix code below

In [30]:
# la_hospital_capacity = la_hospital_capacity.replace(-999999, np.nan)
# la_hospital_capacity["hospital_name"] = [name.lower().title() for name in la_hospital_capacity.hospital_name]
### needs fixing##########
#la_hospital_capacity["percent_beds_used"] = (la_hospital_capacity.total_adult_patients_hospitalized_confirmed_covid_7_day_avg/la_hospital_capacity.total_beds_7_day_avg*100).round(4)
# la_hospital_capacity["percent_icu_beds_used"] = (la_hospital_capacity.icu_beds_used_7_day_sum/la_hospital_capacity.total_icu_beds_7_day_sum*100).round(4)

In [136]:
def translate(column, percent=False, rightMin=5, rightMax=40):
    if percent:
        leftMin = 0
        leftMax = 100
    else:
        leftMin, leftMax = min(column), max(column)
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin
    mapped = []
    
    mapped = (column - leftMin).astype("float")/float(leftSpan)
    mapped = rightMin + mapped*rightSpan

    return mapped

In [137]:
la_hospital_capacity["percent_beds_used_sizes"] = translate(la_hospital_capacity["percent_beds_used"], True)
la_hospital_capacity["percent_icu_beds_used_sizes"] = translate(la_hospital_capacity["percent_icu_beds_used"], True)
la_hospital_capacity["total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum_sizes"] = translate(la_hospital_capacity["total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum"], False, 10, 50)

In [138]:
interest = "percent_beds_used"
la_hospital_capacity['interest'] = la_hospital_capacity[interest]
la_hospital_capacity["sizes"] = la_hospital_capacity[interest+"_sizes"]

dates = list(la_hospital_capacity.groupby("collection_week").groups.keys())
recent_date = sorted(dates)[-1]
income = gdf_tract['income']

Color codes that represent income

In [139]:
blues = ('#BCD2E8', '#91BAD6','#73A5C6','#528AAE','#2E5984','#1E3F66')

In [141]:
multicolor = ('green','orange', 'yellow','red')

In [155]:
columns = ["collection_week", "interest", "x", "y", "sizes", "hospital_name","hospital_subtype", 
            "percent_beds_used", "percent_icu_beds_used","percent_beds_used_sizes", "percent_icu_beds_used_sizes",
            "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum",
                                   "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum_sizes"]

In [159]:
data = {'interest': [interest], "date":[recent_date]}
isource = ColumnDataSource(data=data)

g_df = gdf_tract.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)
colormapper = linear_cmap(field_name='income', palette=blues, low=min(income)-191, high=max(income)) #income mapping

#initializing
p = figure(title="Map", plot_width=1100, plot_height=1000)
p.xaxis.visible = False
p.yaxis.visible = False
chosentile = get_provider(CARTODBPOSITRON)
p.add_tile(chosentile)


#patches###
p.patches('x', 'y', source=gsource, color=colormapper, fill_alpha=1.0, line_color="white", line_width=0.1)
###color bar########
n_ticks = 7
formatter = NumeralTickFormatter(format="$0,0")
ticks = np.linspace(min(income)-191, max(income), n_ticks).round(-1)  # round to desired precision 
color_ticks = FixedTicker(ticks=ticks)
color_bar = ColorBar(color_mapper=colormapper['transform'], width=10,  location=(0,0), ticker=color_ticks, name="Median Income")
color_bar.formatter = formatter
p.add_layout(color_bar, 'right')

###scatter plot##

interest = "percent_beds_used" #initialize variable of interest
p_df = la_hospital_capacity.loc[:, columns]
selection = p_df[p_df['collection_week'] == recent_date]

psource = ColumnDataSource(data=p_df)
curr=ColumnDataSource(data=selection)

callback = CustomJS(args=dict(source=psource, sc=curr, isource=isource), code="""
    var f = cb_obj.value
    sc.data['interest']=[]
    sc.data['sizes']=[]
    
    isource.data["date"] = []
    isource.data["date"].push(f)
    
    var interest = isource.data["interest"][0]
    var sizes = interest + "_sizes"
    
    for(var i = 0; i <= source.get_length(); i++){
        if (source.data['collection_week'][i] == f){
        sc.data['interest'].push(source.data[interest][i])
        sc.data['sizes'].push(source.data[sizes][i])
        }
    }   
    
    isource.change.emit()
    sc.change.emit();
""")

callback_capacity = CustomJS(args=dict(source=psource, sc=curr, isource=isource), code="""
    var f = cb_obj.value
    var dict = {
      "% Beds occupied for covid patients": "percent_beds_used",
      "% ICU occupied used for covid patients": "percent_icu_beds_used",
      "Gross Number of Adult Patients":"total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum"}
    
    var column = dict[f]
    var col_size = column + "_sizes"
    var date = isource.data["date"][0]
    
    isource.data["interest"] = [column]
    
    source.data['interest'] = source.data[column]
    source.data['sizes'] = source.data[col_size]
    
    sc.data['interest']=[]
    sc.data['sizes']=[]
    
    for(var i = 0; i <= source.get_length(); i++){
        if (source.data['collection_week'][i] == date){
        sc.data['interest'].push(source.data[column][i])
        sc.data['sizes'].push(source.data[col_size][i])
        }
    }   
    

    source.change.emit()
    isource.change.emit()
    sc.change.emit();
""")


multimapper = linear_cmap(field_name='interest', palette=multicolor, low=0, high=100)
p.circle(x="x", y="y", color="red", alpha=0.5, name="points", source=curr, size="sizes")

hover = HoverTool(names=["points"])
hover.tooltips = [('Hospital', '@hospital_name'), ('Hospital Type', '@hospital_subtype'), ('Metric of Interest', '@interest{0.2f}')]
p.add_tools(hover)

menu_capacity = Select(options=["% Beds occupied for covid patients", "% ICU occupied used for covid patients",
                "Gross Number of Adult Patients"], value="% Beds occupied for covid patients", title='Capacity')
menu_capacity.js_on_change('value', callback_capacity)

menu = Select(options=dates, value=recent_date, title='Week of Data Collection')
menu.js_on_change('value', callback) # calling the function on change of selection



layout=column(menu, menu_capacity, p) 

show(layout) #uncomment to show plot

outfp = r"covid19map.html"
save(layout, outfp)

'/Users/lucywang/Desktop/data-for-la/COVID-19 Mapping/covid19map.html'

In [15]:
variables = ["collection_week", "interest", "x", "y", "sizes", 
                                    "hospital_name","hospital_subtype", 
                                    "percent_beds_used", "percent_icu_beds_used",
                                   "percent_beds_used_sizes", "percent_icu_beds_used_sizes",
                                   "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum",
                                   "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum_sizes"]

In [20]:
la_hospital_capacity[variables][(la_hospital_capacity.collection_week=="2020-12-25") & (la_hospital_capacity.hospital_name == "Antelope Valley Hospital")]

,collection_week,interest,x,y,sizes,hospital_name,hospital_subtype,percent_beds_used,percent_icu_beds_used,percent_beds_used_sizes,percent_icu_beds_used_sizes,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum_sizes
506,2020-12-25,63.1959,-1.315346e+07,4.121391e+06,27.118565,Antelope Valley Hospital,Short Term,63.1959,84.2342,27.118565,34.48197,930.0,29.892086
